# EDA

## Descriptive statistics

In [ ]:
import pandas as pd
import dtale

df = pd.read_csv('wine.csv')
d = dtale.show(df)
d

In [1]:
import pandas as pd

pd.read_csv('wine_cleared.csv')

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,Portugal,A young and fruity wine that is soft with gent...,Colheita,85,12.000000,Douro,unknown,Roger Voss,@vossroger,Casa de Cambres 2014 Colheita Red (Douro),Portuguese Red,Casa de Cambres
1,Italy,"Aromas of ripe berry, fragrant blue flower and...",unknown,87,35.363389,Tuscany,Morellino di Scansano,Kerin O’Keefe,@kerinokeefe,Frank & Serafico 2012 Morellino di Scansano,Red Blend,Frank & Serafico
2,Spain,Albariño is a fragile style of white wine and ...,unknown,85,34.000000,Galicia,Rías Baixas,Michael Schachner,@wineschach,Lusco 2011 Albariño (Rías Baixas),Albariño,Lusco
3,US,"This blend of 59% Cabernet Sauvignon, 23% Cabe...",Tapteil Vineyard,92,45.000000,Washington,Red Mountain,Sean P. Sullivan,@wawinereport,Cadence 2011 Tapteil Vineyard Red (Red Mountain),Bordeaux-style Red Blend,Cadence
4,Chile,Jammy blueberry and cassis aromas are sturdy b...,Gran Reserva,89,14.000000,Aconcagua Valley,unknown,Michael Schachner,@wineschach,Echeverria 2013 Gran Reserva Malbec (Aconcagua...,Malbec,Echeverria
...,...,...,...,...,...,...,...,...,...,...,...,...
64981,Australia,Another faintly copper-hued Pinot Grigio from ...,Tic Tok,85,14.000000,South Australia,Adelaide Hills,Joe Czerwinski,@JoeCz,James Oatley 2010 Tic Tok Pinot Grigio (Adelai...,Pinot Grigio,James Oatley
64982,Italy,Bellavista's Gran Cuvée Brut offers a pretty f...,Gran Cuvée Brut,90,40.000000,Lombardy,Franciacorta,Kerin O’Keefe,@kerinokeefe,Bellavista 2007 Gran Cuvée Brut Sparkling (Fra...,Sparkling Blend,Bellavista
64983,Chile,The nose on this average SB is bland and neutr...,Cuvée Tradition,85,13.000000,Rapel Valley,unknown,Michael Schachner,@wineschach,Château Los Boldos 2012 Cuvée Tradition Sauvig...,Sauvignon Blanc,Château Los Boldos
64984,US,"Light in texture and color, this is a floral w...",Suacci Vineyard,90,44.000000,California,Sonoma Coast,Virginie Boone,@vboone,Von Holt 2013 Suacci Vineyard Pinot Noir (Sono...,Pinot Noir,Von Holt
